# How to...use DbEngine to execute a simple query

In [1]:
# Sets up the location of the api relative to this notebook 
import sys
sys.path.append('../../../')

In [2]:
# Import the module for connection to a sqllite database
from esgmatching.db_engine.engines.connector_sql_lite import SqlLiteConnector

In [3]:
# Import the module for datasource reflection
from esgmatching.db_data_source.db_data_source import DbDataSource

## 1. Database setup

In [4]:
# Localization of the database to be created in relation to this jupyter notebook
# The database will be created in the /data/dabase folder, under the project main folder
path_db = '../../../tests/data/sqlite/test_esg_matching.db'

In [5]:
# The database connector is represented by the class SqlLiteConnector 
db_conn = SqlLiteConnector()

In [6]:
# The connect() method of the SqlLiteConnector is used to stablish a connection with the database if it exists, 
# or to create a new one. The property path_db defines the location and name of the database.
# The  property show_sql_statement indicates if the SQL statements are echoed (or printed) in the default output channel.
db_conn.path_db = path_db
db_conn.show_sql_statement = False
db_conn.connect()

In [7]:
# Check if the connection was stablished
db_conn.is_connected()

True

## 2. Using DbDataSource to reflect an existent table 

In [8]:
# Create a datasource object
my_table = DbDataSource(db_conn, 'DS_MATCHING')

In [9]:
# Set up important properties before syncronizing the database table
my_table.table_name = 'ESG_MATCHING'

In [10]:
# Sync with database table and bring the attribute names and pks
my_table.sync_with_db_table(set_original_fields=True, set_pks=True)

In [11]:
# Checking the attributes
my_table.get_attribute_names()

[Column('TIMESTAMP', DATETIME(), table=<ESG_MATCHING>, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x000001E2778F25E0>, for_update=False)),
 Column('REF_NAME', VARCHAR(), table=<ESG_MATCHING>),
 Column('TGT_NAME', VARCHAR(), table=<ESG_MATCHING>),
 Column('MATCHING_TYPE', VARCHAR(), table=<ESG_MATCHING>),
 Column('MATCHING_SCOPE', VARCHAR(), table=<ESG_MATCHING>),
 Column('MATCHING_RULE', VARCHAR(), table=<ESG_MATCHING>),
 Column('REF_ID', INTEGER(), table=<ESG_MATCHING>),
 Column('REF_COMPANY', VARCHAR(), table=<ESG_MATCHING>),
 Column('REF_COUNTRY', VARCHAR(), table=<ESG_MATCHING>),
 Column('TGT_ID', INTEGER(), table=<ESG_MATCHING>),
 Column('TGT_COMPANY', VARCHAR(), table=<ESG_MATCHING>),
 Column('TGT_COUNTRY', VARCHAR(), table=<ESG_MATCHING>),
 Column('ISIN', VARCHAR(), table=<ESG_MATCHING>),
 Column('LEI', VARCHAR(), table=<ESG_MATCHING>),
 Column('SEDOL', VARCHAR(), table=<ESG_MATCHING>)]

## 3. Build a query

In [12]:
# Import the query builder module 
from esgmatching.db_engine.builders.select_builder import SelectBuilder
from esgmatching.db_engine.builders.sql_condition_builder import SqlConditionBuilder
from esgmatching.db_engine.executor.dql_manager import DqlManager

In [13]:
# Instantiate the condition builder as to construct where clauses for your select statement
condition_builder = SqlConditionBuilder()

In [14]:
# Get the reference to the column in the datasource to be used in the where clause
col_tgt = my_table.get_table_column('REF_ID')
col_tgt

Column('REF_ID', INTEGER(), table=<ESG_MATCHING>)

In [15]:
# Build the where condition clause
where_condition = condition_builder.create_condition().equal_value(col_tgt, 1).get_condition()
print(str(where_condition))

"ESG_MATCHING"."REF_ID" = :REF_ID_1


In [16]:
# Instantiate the select builder as to construct select statements
sql_builder = SelectBuilder()

In [17]:
# Define the names of the column for your select and get the table reference to them 
col_names = ['REF_ID', 'REF_COMPANY', 'REF_COUNTRY', 'TGT_ID', 'TGT_COMPANY', 'TGT_COUNTRY', 'ISIN', 'LEI', 'SEDOL']
col_alias = ['REF_ID', 'REF_ENTITY', 'REF_COUNTRY', 'TGT_ID', 'TGT_ENTITY', 'TGT_COUNTRY', 'ISIN_ID', 'LEI_ID', 'SEDOL_ID']
cols_db = my_table.get_table_columns(col_names, col_alias)

In [18]:
# Uncomment the next line to select the columns without an alias
# cols_db = my_table.get_table_columns(col_names)

In [19]:
# Build the sql statement
sql_stm = sql_builder.create_select(cols_db).from_table(my_table.table_obj).where_condition(where_condition).build_statement()
print(str(sql_stm))

SELECT "ESG_MATCHING"."REF_ID", "ESG_MATCHING"."REF_COMPANY", "ESG_MATCHING"."REF_COUNTRY", "ESG_MATCHING"."TGT_ID", "ESG_MATCHING"."TGT_COMPANY", "ESG_MATCHING"."TGT_COUNTRY", "ESG_MATCHING"."ISIN", "ESG_MATCHING"."LEI", "ESG_MATCHING"."SEDOL" 
FROM "ESG_MATCHING" 
WHERE "ESG_MATCHING"."REF_ID" = ?


In [20]:
# Create the Data Query Language (DQL) manager object that allows to execute queries in the database
sql_executor = DqlManager(db_conn)

In [21]:
# Execute the query and receive data as a list of tuples
result_data = sql_executor.query_by_sql_statement(sql_stm)
result_data

[(1, 'CENTRAL PERK', 'SK', 1, 'CENTRAL PERK AND SONS', 'DE', 'SK1120005824', '097900BHK10000084115', None),
 (1, 'CENTRAL PERK', 'SK', 7, 'CENTRAL PERK', 'SK', 'SK1120005824', None, 'B1YW440')]

In [22]:
# Execute the query and receive data as a pandas dataframe
result_data = sql_executor.query_by_sql_statement(sql_stm, as_pandas_df=True)
result_data

,REF_ID,REF_COMPANY,REF_COUNTRY,TGT_ID,TGT_COMPANY,TGT_COUNTRY,ISIN,LEI,SEDOL
0,1,CENTRAL PERK,SK,1,CENTRAL PERK AND SONS,DE,SK1120005824,097900BHK10000084115,None
1,1,CENTRAL PERK,SK,7,CENTRAL PERK,SK,SK1120005824,None,B1YW440
